<div align="center">

# Portfolio Optimization 
<h4>
  Wesley Dyk<br>
  <small style="font-weight: normal;">
    Senior Quantum Solutions Architect<br>
    Quantum Computing Inc.
  </small>
</h4>

<br>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qci-wdyk/eqc-models-tutorial/blob/main/tutorial07-portfolio-optimization.ipynb)

</div>


Select a value for $\xi$ and solve the program
$$
\begin{equation}
    \min_{\{w_{i}\}_{i \in \{1, 2,..., K\}}} [-E(R) R_{B} + \xi \mathrm{VAR}(R)]
\end{equation}
$$

to select a weighted proportion of the portfolio to seek maximum return and minimized risk.


In [1]:
!pip install eqc-models
import os
import sys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from eqc_models.allocation import PortMomentum
from eqc_models.solvers import Dirac3ContinuousCloudSolver
from utils import (
    get_nasdaq100_constituents,
    get_port_stats,
)
try:
    from google.colab import userdata
except ImportError:
    userdata = None


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Define the API URL and token  for QCI
api_url ="https://api.qci-prod.com"
if userdata is None:
    api_token = "" # replace or use environment variables to configure
else:
    api_token = userdata.get("QCI_TOKEN")
    os.environ["QCI_TOKEN"] = api_token
    os.environ["QCI_API_URL"] = api_url

In [ ]:
# Set parameters
ADJ_DATE = "2022-01-01"
LOOKBACK_DAYS = 60
LOOKFORWARD_DAYS = 30

# Get stock list
stocks = get_nasdaq100_constituents(
    ADJ_DATE, LOOKBACK_DAYS, LOOKFORWARD_DAYS,
)

# Get portfolio model
model = PortMomentum(
    stocks=stocks,
    adj_date=ADJ_DATE,
    stock_data_dir="stock_prices",
    lookback_days=LOOKBACK_DAYS,
    window_days=30,
    window_overlap_days=15,
    weight_upper_limit=0.08,
    r_base=0.05 / 365,
    alpha=5.0, # penalty multiplier term
    beta=1.0, # penalty multiplier term
    xi=1.0, # multiplier as in the formula
)

Chose 101 of 102 stocks


In [4]:
# Solve on Dirac-3
solver = Dirac3ContinuousCloudSolver()
response = solver.solve(
    model,
    sum_constraint=100,
    relaxation_schedule=2,
    num_samples=10,
)
sol = response["results"]["solutions"][0][:len(stocks)]

print(response)

weight_hash = {}
for i in range(len(stocks)):
    weight_hash[stocks[i]] = sol[i] / 100.0

tot_weight = sum(weight_hash.values())

if tot_weight != 1.0:
    for stock in stocks:
        weight_hash[stock] = weight_hash[stock] / tot_weight

weight_df = pd.DataFrame(
    {
        "Stock": [item for item in weight_hash.keys()],
        "Allocation": [
            weight_hash[item] for item in weight_hash.keys()
        ],
    }
)
weight_df["Date"] = ADJ_DATE
weight_df = weight_df[weight_df["Allocation"] > 0]

ret_df = get_port_stats(weight_df, 30)

print(ret_df)


2025-08-28 22:15:56 - Dirac allocation balance = 0 s (unmetered)
2025-08-28 22:15:56 - Job submitted: job_id='68b1297c8060c933979636a4'
2025-08-28 22:15:56 - QUEUED
2025-08-28 22:15:59 - RUNNING
2025-08-28 22:16:17 - COMPLETED
2025-08-28 22:16:19 - Dirac allocation balance = 0 s (unmetered)
{'job_info': {'job_id': '68b1297c8060c933979636a4', 'job_submission': {'problem_config': {'normalized_qudit_hamiltonian_optimization': {'polynomial_file_id': '68b1297bacc178773e9a5307'}}, 'device_config': {'dirac-3_normalized_qudit': {'num_samples': 10, 'relaxation_schedule': 2, 'sum_constraint': 100}}}, 'job_status': {'submitted_at_rfc3339nano': '2025-08-29T04:15:56.651Z', 'queued_at_rfc3339nano': '2025-08-29T04:15:56.651Z', 'running_at_rfc3339nano': '2025-08-29T04:15:57.627Z', 'completed_at_rfc3339nano': '2025-08-29T04:16:16.231Z'}, 'job_result': {'file_id': '68b12990acc178773e9a530d', 'device_usage_s': 16}}, 'status': 'COMPLETED', 'results': {'counts': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'energies': 